# Code Mixed Sentiment Analysis

With respect to the mixed dataset containing Hindi and English words, we propose a neural model for classification of sentiments as "positive", "neutral" or "negative"

## Imports and Data Acquisition

We use pandas for reading the tab separated raw files; while
numpy is used for all numerical operations

We setup drive access to colab to load the raw data files for testing and training

In [0]:
import pandas as pd 
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


TSV values can be read as csv with a separator parameter as `\t`. This is then converted into a numpy array for easier processing

In [0]:
train_data = pd.read_csv('drive/My Drive/nlp assignment 3/train.txt',sep='\t',names = ["Meta", "UID", "Sent"])
test_data = pd.read_csv('drive/My Drive/nlp assignment 3/test.txt',sep='\t',names = ["Meta", "UID", "Sent"])

In [0]:
train_data = np.array(train_data)
test_data = np.array(test_data)

### Parsing data

In [0]:
token_train = []
lang_train = []
y_train = []
uid_train = []

token_buf = []
lang_buf = []

for i in range(train_data.shape[0]):
  if(train_data[i][0]!='meta' and type(train_data[i][0]) == str):
    token_buf.append(train_data[i][0].lower())
    if (train_data[i][1] == 'Hin'):
      lang_buf.append(1)
    else:
      lang_buf.append(0)
  else:
    if(i!=0):
      token_train.append(token_buf)
      lang_train.append(lang_buf)

    token_buf = []
    lang_buf = []

    uid_train.append(train_data[i][1])
    if(train_data[i][2] == 'positive'):
      y_train.append(1)
    elif(train_data[i][2] == 'negative'):
      y_train.append(-1)
    else:
      y_train.append(0)

token_train.append(token_buf)
lang_train.append(lang_buf)

In [6]:
token_train = np.array(token_train)
lang_train = np.array(lang_train)

print("Sanity Checks for loading training data")
print("Shape of tweets array:",token_train.shape)
print("Shape of language masks",lang_train.shape)
print("Number of labels",len(y_train))
print("Number of uids:",len(uid_train))

Sanity Checks for loading training data
Shape of tweets array: (13541,)
Shape of language masks (13541,)
Number of labels 13541
Number of uids: 13541


Loading test data as a numpy array

In [0]:
token_test = []
lang_test = []
y_test = []
uid_test = []

token_buf = []
lang_buf = []

for i in range(test_data.shape[0]):
  if(test_data[i][0]!='meta' and type(test_data[i][0]) == str):
    token_buf.append(test_data[i][0].lower())
    if (test_data[i][1] == 'Hin'):
      lang_buf.append(1)
    else:
      lang_buf.append(0)
  else:
    if(i!=0):
      token_test.append(token_buf)
      lang_test.append(lang_buf)

    token_buf = []
    lang_buf = []

    uid_test.append(test_data[i][1])
    if(test_data[i][2] == 'positive'):
      y_test.append(1)
    elif(test_data[i][2] == 'negative'):
      y_test.append(-1)
    else:
      y_test.append(0)

token_test.append(token_buf)
lang_test.append(lang_buf)

In [8]:
token_test = np.array(token_test)
lang_test = np.array(lang_test)

print("Sanity Checks for loading testing data")
print("Shape of tweets array:",token_test.shape)
print("Shape of language masks",lang_test.shape)
print("Number of labels",len(y_test))
print("Number of uids:",len(uid_test))

Sanity Checks for loading testing data
Shape of tweets array: (1872,)
Shape of language masks (1872,)
Number of labels 1872
Number of uids: 1872


## **Multilingual BERT + MLP**

This model is based on BERT embeddings. For multilingual data, google provides support across multiple languages. We use these weights for Multilingual BERT and fine tune the network with respect to our data.<br>

This is then classified using a MLP based model

### Basic Imports

We need tensorflow for the model and tensorflow_hub for the weights. Other libraries are used for helper functions, convenience or evaluations

In [9]:
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
tf.get_logger().setLevel('INFO')  # 'INFO' for including Information

import tensorflow_hub as hub
from datetime import datetime

from sklearn.model_selection import train_test_split

### Getting BERT

We run a `pip3 install bert-tensorflow` in case BERT does not exist 

Next, we import bert and its classifier, optimizer and tokenizer for fine tuning the model

In [10]:
!pip3 install bert-tensorflow

In [11]:
import bert
from bert import run_classifier, optimization, tokenization

Join all tokens as separate tweets and store their corresponding labels to create a dataframe

In [0]:
train = []
for i in range(len(token_train)):
  train.append(' '.join(token_train[i]))
train = pd.DataFrame(train,columns = ['Sentences'])
train['Label'] = y_train

In [0]:
test = []
for i in range(len(token_test)):
  test.append(' '.join(token_test[i]))
test = pd.DataFrame(test,columns = ['Sentences'])
test['Label'] = y_test

BERT functions by finding closer words such that <i>input + A - B = output</i> 

The code below creates data in a way which BERT understands

In [0]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x['Sentences'], 
                                                                   text_b = None, 
                                                                   label = x['Label']), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['Sentences'], 
                                                                   text_b = None, 
                                                                   label = x['Label']), axis = 1)


In [16]:
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


### Label assignment

We assign all positive as "1", negative as "-1" and neutral as "0"

In [0]:
label_list = [-1,0,1]

### Model Params
We set max sequence length to be `256` and supply data to generate features for BERT classifier

In [18]:
MAX_SEQ_LENGTH = 256

# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 13541


INFO:tensorflow:Writing example 0 of 13541


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 2)


INFO:tensorflow:label: 1 (id = 2)


In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""


  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  bert_output_layer = bert_outputs["pooled_output"]

  # Task dedicated MLP

  hidden_size = bert_output_layer.shape[-1].value
  print("BERT O/P Size",hidden_size)

  h1_nodes = 512
  h2_nodes = 256

  h1 = tf.layers.dense(bert_output_layer, h1_nodes , activation=tf.nn.relu)
  output_layer = tf.layers.dense(h1, h2_nodes , activation=tf.nn.relu)
  
  output_weights = tf.get_variable(
      "output_weights", [num_labels, h2_nodes],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    # Equivalent to categorical crossentropy
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, predicted_labels, log_probs)

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        
        # Function to find confusion matrix
        def eval_confusion_matrix(label_ids, predicted_labels):
          with tf.variable_scope("eval_confusion_matrix"):
            conf_matrix = tf.confusion_matrix(labels=label_ids, predictions=predicted_labels, num_classes=3)

            conf_matrix_sum = tf.Variable(tf.zeros(shape=(3,3), dtype=tf.int32),
                                            trainable=False,
                                            name="confusion_matrix_result",
                                            collections=[tf.GraphKeys.LOCAL_VARIABLES])


            update_op = tf.assign_add(conf_matrix_sum, conf_matrix)

            return tf.convert_to_tensor(conf_matrix_sum), update_op
     
        return {
            "eval_accuracy": accuracy,
            # "auc": auc,
            "precision": precision,
            "recall": recall,
            "confusion_matrix": eval_confusion_matrix(
            label_ids,
            predicted_labels)
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [0]:
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
# Warmup is a period of time where the learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.2
# Model configs
SAVE_CHECKPOINTS_STEPS = 200
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify output directory and number of checkpoint steps to save
OUTPUT_DIR = 'NLP Assignment III'
DO_DELETE = True  # To clear previous weights; set false for retaining them

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)

In [0]:
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [65]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'NLP Assignment III', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 200, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f43c13895c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'NLP Assignment III', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 200, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f43c13895c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [77]:
print(f'Begin Training')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Begin Training
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


BERT O/P Size 768


...

Instructions for updating:
Use standard file APIs to delete files with this prefix.


Instructions for updating:
Use standard file APIs to delete files with this prefix.


INFO:tensorflow:Loss for final step: 0.92375803.


INFO:tensorflow:Loss for final step: 0.92375803.


Training took time  0:03:28.517654


In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [84]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


BERT O/P Size 768


...

INFO:tensorflow:Summary for np.ndarray is not visible in Tensorboard by default. Consider using a Tensorboard plugin for visualization (see https://github.com/tensorflow/tensorboard-plugin-example/blob/master/README.md for more information).


INFO:tensorflow:Summary for np.ndarray is not visible in Tensorboard by default. Consider using a Tensorboard plugin for visualization (see https://github.com/tensorflow/tensorboard-plugin-example/blob/master/README.md for more information).


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 846: NLP Assignment III/model.ckpt-846


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 846: NLP Assignment III/model.ckpt-846


{'confusion_matrix': array([[366, 126,  41],
        [253, 333, 171],
        [ 97, 151, 334]], dtype=int32),
 'eval_accuracy': 0.5518162,
 'global_step': 846,
 'loss': 0.9703928,
 'precision': 0.85553634,
 'recall': 0.7386109}

In [0]:
def report (conf_mat):
  d = len(conf_mat)
  hor = []
  ver = []
  for i in range(d):
    hor.append(np.sum(conf_mat[i,:]))
    ver.append(np.sum(conf_mat[:,i]))
  pre = []
  rec = []
  f1 = []
  for i in range(d):
    pre.append(conf_mat[i,i]/ver[i])
    rec.append(conf_mat[i,i]/hor[i])
    f1.append(2*rec[i]*pre[i]/(pre[i]+rec[i]))

  return pre,rec,f1



In [0]:
conf_mat = np.array([[366, 126,  41],
        [253, 333, 171],
        [ 97, 151, 334]])

In [87]:
report(conf_mat)

([0.5111731843575419, 0.5459016393442623, 0.6117216117216118],
 [0.6866791744840526, 0.43989431968295906, 0.5738831615120275],
 [0.5860688550840673, 0.487198244330651, 0.5921985815602837])